In [ ]:
from dolfinx import fem, default_scalar_type, mesh as msh
import dolfinx
import ufl
from SPDE_problems import *
import torch
from torch_geometric.data import Data
import torch
from mpi4py import MPI
import pyvista as pv
from utils.plotter import fem_plotter_grid


def int_to_prblm(idx, mesh):
    if idx == 0:
        return wedge(mesh=mesh)
    if idx == 1:
        return bump(mesh=mesh)
    if idx == 2:
        return lifted_edge(mesh=mesh)
    if idx == 3:
        return cylinder(mesh=mesh)
    if idx == 4:
        return falloff(mesh=mesh)
    if idx == 5:
        return curved_wave(mesh=mesh)
    if idx == 6:
        return curved_waves(mesh=mesh)

In [1]:
import os
import torch
from torch_geometric.data import InMemoryDataset, Data
from torch_geometric.loader import DataLoader

class graph_dataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0], weights_only=False)

    @property
    def raw_file_names(self):
        return [f for f in os.listdir(self.raw_dir) if f.endswith(".pt")]

    @property
    def processed_file_names(self):
        return ["data.pt"]

    def download(self):
        pass

    def process(self):
        data_list = []

        for raw_path in self.raw_paths:
            data = torch.load(raw_path, weights_only=False)
            
            assert data.x is not None
            assert data.edge_index is not None

            data_list.append(data)

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)

        torch.save((data, slices), self.processed_paths[0])

    
train_set = graph_dataset(root="data/training_set/input_values/")
test_set = graph_dataset(root="data/test_set/input_values/")

In [2]:
from torch_geometric.loader import DataLoader
import torch_geometric as tg

batch_size = 5
train_loader = DataLoader(train_set, batch_size=batch_size)
test_loader = DataLoader(test_set, batch_size=batch_size)


In [ ]:
class MLP(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = torch.nn.linear(10, 64)
        self.conv2 = torch.nn.linear(64, 32)
        self.conv3 = torch.nn.linear(32, 32)
        self.conv4 = torch.nn.linear(32, 1)

    def forward(self, data) -> torch.Tensor:
        x, edge_index = data.x, data.edge_index

        # Perform two-layers of message passing:
        h = self.conv1(x=x, edge_index=edge_index)
        h = h.relu()
        h = self.conv2(x=h, edge_index=edge_index)
        h = h.relu()
        h = self.conv3(x=h, edge_index=edge_index)
        h = h.relu()
        h = self.conv4(x=h, edge_index=edge_index)

        return h

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = tg.nn.conv.GCNConv(10, 64)
        self.conv2 = tg.nn.conv.GCNConv(64, 32)
        self.conv3 = tg.nn.conv.GCNConv(32, 32)
        self.conv4 = tg.nn.conv.GCNConv(32, 1)

    def forward(self, data) -> torch.Tensor:
        x, edge_index = data.x, data.edge_index

        # Perform two-layers of message passing:
        h = self.conv1(x=x, edge_index=edge_index)
        h = h.relu()
        h = self.conv2(x=h, edge_index=edge_index)
        h = h.relu()
        h = self.conv3(x=h, edge_index=edge_index)
        h = h.relu()
        h = self.conv4(x=h, edge_index=edge_index)

        return h
    
class SAGE(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = tg.nn.conv.SAGEConv(10, 64)
        self.conv2 = tg.nn.conv.SAGEConv(64, 32)
        self.conv3 = tg.nn.conv.SAGEConv(32, 32)
        self.conv4 = tg.nn.conv.SAGEConv(32, 1)

    def forward(self, data) -> torch.Tensor:
        x, edge_index = data.x, data.edge_index

        # Perform two-layers of message passing:
        h = self.conv1(x=x, edge_index=edge_index)
        h = h.relu()
        h = self.conv2(x=h, edge_index=edge_index)
        h = h.relu()
        h = self.conv3(x=h, edge_index=edge_index)
        h = h.relu()
        h = self.conv4(x=h, edge_index=edge_index)

        return h
    
    
class GAT(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = tg.nn.conv.GATConv(10, 64)
        self.conv2 = tg.nn.conv.GATConv(64, 32)
        self.conv3 = tg.nn.conv.GATConv(32, 32)
        self.conv4 = tg.nn.conv.GATConv(32, 1)

    def forward(self, data) -> torch.Tensor:
        x, edge_index = data.x, data.edge_index

        # Perform two-layers of message passing:
        h = self.conv1(x=x, edge_index=edge_index)
        h = h.relu()
        h = self.conv2(x=h, edge_index=edge_index)
        h = h.relu()
        h = self.conv3(x=h, edge_index=edge_index)
        h = h.relu()
        h = self.conv4(x=h, edge_index=edge_index)

        return h


    
class GATv2(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = tg.nn.conv.GATv2Conv(10, 64)
        self.conv2 = tg.nn.conv.GATv2Conv(64, 32)
        self.conv3 = tg.nn.conv.GATv2Conv(32, 32)
        self.conv4 = tg.nn.conv.GATv2Conv(32, 1)

    def forward(self, data) -> torch.Tensor:
        x, edge_index = data.x, data.edge_index

        # Perform two-layers of message passing:
        h = self.conv1(x=x, edge_index=edge_index)
        h = h.relu()
        h = self.conv2(x=h, edge_index=edge_index)
        h = h.relu()
        h = self.conv3(x=h, edge_index=edge_index)
        h = h.relu()
        h = self.conv4(x=h, edge_index=edge_index)

        return h
def train(model, loader, optimizer, device):
    #model.train()
    total_loss = 0

    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()

        out = model(data)
        loss = F.mse_loss(out, data.y.view(-1,1))

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)
model = GCN()

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer)

import torch
import torch.nn.functional as F

def train(model, loader, optimizer, device):
    #model.train()
    total_loss = 0

    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()

        out = model(data)
        loss = F.mse_loss(out, data.y.view(-1,1))

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)


In [ ]:
for i in range(5000):
    print(i)
    loss = train(model=model, loader=train_loader, optimizer=optimizer, device='cpu')
    print(loss)
    scheduler.step(loss)

0
0.2721844404521916
1
0.27217949202491176
2
0.2721763863745663
3
0.2721731346100569
4
0.27216761993865174
5
0.272164150658581
6
0.27215966561602223
7
0.2721570138302114
8
0.27215085675319034
9
0.2721489146351814
10
0.27214543769756955
11
0.272139234882262
12
0.2721375038640367
13
0.2721329695648617
14
0.2721287202503946
15
0.27212599230309326
16
0.2721211765375402
17
0.2721166985316409
18
0.27211343662606347
19
0.2721086192048258
20
0.2721049642811219
21
0.27210135613050723
22
0.2720960235844056
23
0.2720940951257944
24
0.2720873800830709
25
0.27208706322643494
26
0.27207993653913337
27
0.27207734083963764
28
0.27207403174704975
29
0.2720692795183923
30
0.27206580133901703
31
0.27206474335657227
32
0.2720569223165512
33
0.272056485629744
34
0.2720515796293815
35
0.2720482817126645
36
0.27204243735306793
37
0.272044163197279
38
0.2720362939354446
39
0.27203475890888107
40
0.2720309959517585
41
0.2720268706066741
42
0.2720251515921619
43
0.2720202294488748
44
0.27201721465422046
45
0.27